# Advent of Code 2022 - Day 9

In [20]:
from pathlib import Path
from typing import Final, List, Dict, Tuple, Set, Any, TypeAlias

## Part One

- [X] Read input
- [ ] Split each instruction to a Tuple of (Direction, Number)
- [ ] Create a grid for the rope to move on?
- [ ] Move Head
- [ ] Compare Tail position to head and move accordingly
- [ ] Mark position of tail on a boolean grid (True is tail has passed that space)
- [ ] Count grid


In [21]:
INPUT_NAME : str = "day9_input_test.txt"
MovePosition : TypeAlias = Tuple[int, int]

In [8]:
def read_input(input_name: str) -> List[str]:
    input_path : Path = Path.cwd() / "input" / input_name

    with open(input_path, "r") as input_file:

        lines : List = input_file.readlines()

    return [line.rstrip("\n") for line in lines]

In [9]:
instructions_input : List[MovePosition]= [tuple(line.split(" ")) for line in read_input(INPUT_NAME)]

In [11]:

STARTING_POSITION : Final[MovePosition]= (0,0)
STEP = 1

In [13]:
covered_positions : List[MovePosition] = []

In [28]:
def move_head(current_position: MovePosition, move: MovePosition) -> MovePosition:
    """
    Co-ordinates are relative to start (0,0), as (row, col)
    """
    ROW = 0
    COL = 1
    
    return (current_position[ROW] + move[ROW], current_position[COL] + move[COL])
    

In [35]:
def move_tail(head_position: MovePosition, current_position: MovePosition) -> MovePosition:
    
    

In [38]:
def make_move(
    direction: str, count: str, head_position: MovePosition, tail_position: MovePosition
) -> Tuple[MovePosition, List[MovePosition]]:
    """
    Returns current head position, and list of positions tail has been in for this move
    """

    count = int(count)

    step = -1 if direction in ["L", "D"] else 1
    
    current_head_position = head_position
    current_tail_position = tail_position
    covered_positions = []

    for idx in range(count):
        if direction in ["L", "R"]:
            move : MovePosition = (0, step)
        elif direction in ["U", "D"]:
            move = (step, 0)
        current_head_position = move_head(current_head_position, move)
        current_tail_position = move_tail(current_head_position, current_tail_position)
        
    return current_head_position
        